In [8]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, './../src')

import data_loader
from energy_channels import EnergyChannel
import enhancement_finder
import datetime
import matplotlib.pyplot as plt
import plot_tools
import numpy as np
import pandas as pd

import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(enhancement_finder)

%matplotlib qt

In [9]:
start = datetime.datetime(year = 2013, month = 1, day = 1)
end = datetime.datetime(year = 2014, month = 1, day = 1)

dataRefsA = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)
dataRefsB = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_A, L_A, mlt_A, epoch_A, energies = dataRefsA.get_all_data()
fesa_B, L_B, mlt_B, epoch_B, energies = dataRefsB.get_all_data()

omni = data_loader.load_omni_data(start=start, end=end)


Loading REPT data between: 2013-01-01 00:00:00 and 2014-01-01 00:00:00.
Loading : REPT_201301_A.npz
Loading : REPT_201302_A.npz
Loading : REPT_201303_A.npz
Loading : REPT_201304_A.npz
Loading : REPT_201305_A.npz
Loading : REPT_201306_A.npz
Loading : REPT_201307_A.npz
Loading : REPT_201308_A.npz
Loading : REPT_201309_A.npz
Loading : REPT_201310_A.npz
Loading : REPT_201311_A.npz
Loading : REPT_201312_A.npz
Loading : REPT_201401_A.npz
Loading REPT data between: 2013-01-01 00:00:00 and 2014-01-01 00:00:00.
Loading : REPT_201301_B.npz
Loading : REPT_201302_B.npz
Loading : REPT_201303_B.npz
Loading : REPT_201304_B.npz
Loading : REPT_201305_B.npz
Loading : REPT_201306_B.npz
Loading : REPT_201307_B.npz
Loading : REPT_201308_B.npz
Loading : REPT_201309_B.npz
Loading : REPT_201310_B.npz
Loading : REPT_201311_B.npz
Loading : REPT_201312_B.npz
Loading : REPT_201401_B.npz
Loading OMNI data between: 2013-01-01 00:00:00 and 2014-01-01 00:00:00
Loaded OMNI Data for : 2013-01-01 00:00:00
Loaded OMNI Da

In [10]:
l_cut = 5.5
tol = 0.1

satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 6.5)]
fesa_B = fesa_B[:, (energies < 6.5)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

In [11]:
grid_t = "6h"

fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
#FESA = pd.concat([FESA_A_DF, FESA_B_DF], sort=False).sort_index()

#FESA = FESA.rolling(window=avg_t, center=True).mean()

In [17]:

slow_moving = omni["AE"].rolling("3h", center=False).mean()
#f = (OMNI["AE"] - slow_moving)
f = omni["AE"].diff()
threshold = 25
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("3h", center=False).sum().rolling("12h", center=True).mean()

fig, axs = plt.subplots(4, 1, sharex="all")

num_energies = fesa.shape[1]
colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=colors(E), linewidth=2.5)
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[1].set_ylabel("# Crossing Threshold")
axs[1].set_ylim(bottom=0)
#axs[1].hlines(y=100, xmin=start, xmax=end, color="green", label="Enhancement Trigger")


axs[2].plot(omni.index, f, color="black", label="$|{\delta}AE|$")
axs[2].set_ylabel("$|{\delta}AE|$ (nT)")
axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
axs[2].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")

#axs[2].set_ylim(bottom=0)
axs[2].legend()

axs[3].yaxis.grid(color='gray', linestyle='dashed')
axs[3].xaxis.grid(color='gray', linestyle='dashed')
axs[3].plot(omni.index, omni["AE"], color="black", label="AE")
axs[3].plot(omni.index, slow_moving, color="blue", linewidth=2, label="Slow Average")
axs[3].set_ylabel("AE (nT)")
axs[3].set_xlabel("Time")
axs[3].set_ylim(bottom=0)
axs[3].legend()

fig.set_size_inches(12, 8)

plt.show()

<>:37: SyntaxWarning: invalid escape sequence '\d'
<>:38: SyntaxWarning: invalid escape sequence '\d'
<>:37: SyntaxWarning: invalid escape sequence '\d'
<>:38: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\3032327977.py:37: SyntaxWarning: invalid escape sequence '\d'
  axs[2].plot(omni.index, f, color="black", label="$|{\delta}AE|$")
C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\3032327977.py:38: SyntaxWarning: invalid escape sequence '\d'
  axs[2].set_ylabel("$|{\delta}AE|$ (nT)")
C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\3032327977.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("plasma", num_energies)


In [18]:
above_zero = omni["Bz"] > 0
below_zero = omni["Bz"] <= 0


f = omni["Bz"].diff()
threshold = 1
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("3h", center=False).sum()

fig, axs = plt.subplots(4, 1, sharex="all")

num_energies = fesa.shape[1]
colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=colors(E), linewidth=2.5)


axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[1].set_ylabel("# Crossing Threshold")
axs[1].set_ylim(bottom=0)

axs[2].plot(omni.index, f, color="black")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].set_ylabel("${\delta}B_{z}$ (nT)")
axs[2].hlines(y=0, xmin=start, xmax=end, color="black")
axs[2].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
axs[2].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")

axs[3].plot(omni.index, omni["Bz"], color="black")

#axs[3].plot(OMNI.index[above_zero], OMNI["Bz"][above_zero], ".", color="blue", markersize=3.0)
#axs[3].plot(OMNI.index[below_zero], OMNI["Bz"][below_zero], ".", color="red", markersize=3.0)
axs[3].yaxis.grid(color='gray', linestyle='dashed')
axs[3].xaxis.grid(color='gray', linestyle='dashed')
axs[3].set_ylabel("$B_{z}$ (nT)")
plt.show()



<>:42: SyntaxWarning: invalid escape sequence '\d'
<>:42: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\4133237877.py:42: SyntaxWarning: invalid escape sequence '\d'
  axs[2].set_ylabel("${\delta}B_{z}$ (nT)")
C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\4133237877.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("plasma", num_energies)


In [19]:
f = omni["Bz"].diff()
threshold = 0.80
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings_BZ = zero_crossings.rolling("3h", center=False).sum().rolling("12h", center=True).mean()

f = omni["AE"].diff()
threshold = 20
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings_AE = zero_crossings.rolling("3h", center=False).sum().rolling("12h", center=True).mean()


fig, axs = plt.subplots(2, 1, sharex="all")

num_energies = fesa.shape[1]
colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=colors(E), linewidth=2.5)
    
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")


axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].plot(num_crossings_AE.index, num_crossings_AE, color="Red", label="AE")
axs[1].plot(num_crossings_BZ.index, num_crossings_BZ, color="Blue", label="Bz")
axs[1].set_ylabel("# Crossings")
axs[1].set_ylim(bottom=0)
axs[1].legend()

C:\Users\brofe\AppData\Local\Temp\ipykernel_11764\3382380090.py:25: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("plasma", num_energies)
